Connect our Jupyter Notebook to our database file:

In [2]:


%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Write a query to return information on the tables in the database.

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


write and run another query that returns the first 5 rows of the facts table in the database.

In [4]:
%%sql
select * from facts limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Write a single query that returns the:

Minimum population
Maximum population
Minimum population growth
Maximum population growth

In [5]:
%%sql
select min(population), 
max(population),
min(population_growth),
max(population_growth)
from facts

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,7256490011,0.0,4.02


Write a query that returns the countrie(s) with the minimum population.

In [6]:
%%sql
select name from facts where population IN (select min(population) from facts);


Done.


name
Antarctica


Write a query that returns the countrie(s) with the maximum population.

In [7]:
%%sql
select name from facts where population IN (select max(population) from facts);

Done.


name
World


Recompute the summary statistics you found earlier while excluding the row for the whole world. They were:

1. Minimum population
2. Maximum population
3. Minimum population growth
4. Maximum population growth

In [8]:
%%sql
select min(population), 
max(population),
min(population_growth),
max(population_growth)
from facts
where name <>'World'

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,1367485388,0.0,4.02


calculate the average value for the following columns:

population
area

In [19]:
%%sql
SELECT AVG(population), 
AVG(area)
from facts
 WHERE name <> ' World';

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


Write a query that finds all countries meeting both of the following conditions:
The population is above average.
The area is below average.

In [10]:
%%sql
select * 
from facts
where population > (select AVG(population) from facts)
and area <( select AVG(area) from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


What country has the most people? What country has the highest growth rate?

In [17]:
%%sql
select name, population 
from facts 
where population == (select max(population) 
                     from facts 
                     where name <>'World')

Done.


name,population
China,1367485388


In [12]:
%%sql
select name, population_growth
from facts 
where population_growth == (select max(population_growth) 
                     from facts 
                     where name <>'World')

Done.


name,population_growth
South Sudan,4.02


Which countries have the highest ratios of water to land? Which countries have more water than land?

In [13]:
%%sql
select name,CAST(area_water as float)/ area_land as water_land_ratio 
from facts 
order by water_land_ratio desc
limit 10

Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


In [15]:
%%sql
select name 
from facts 
where area_land< area_water

Done.


name
British Indian Ocean Territory
Virgin Islands


Which countries will add the most people to their population next year?

In [37]:
%%sql
select name, population, population_growth,
CAST ( population_growth/100 * population as INT)  as new_population
from facts
where name <> ('World')
order by new_population desc
limit 10

Done.


name,population,population_growth,new_population
India,1251695584,1.22,15270686
China,1367485388,0.45,6153684
Nigeria,181562056,2.45,4448270
Pakistan,199085847,1.46,2906653
Ethiopia,99465819,2.89,2874562
Bangladesh,168957745,1.6,2703323
United States,321368864,0.78,2506677
Indonesia,255993674,0.92,2355141
"Congo, Democratic Republic of the",79375136,2.45,1944690
Philippines,100998376,1.61,1626073


Which countries have a higher death rate than birth rate?

In [29]:
%%sql
select name, death_rate, birth_rate
from facts 
where death_rate > birth_rate
order by death_rate desc

Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51


What countries have the highest population/area ratio and how does it compare to list we found in the previous screen?

In [26]:
%%sql
select name, population, area, population/ area as ratio
from facts
order by ratio desc
 

Done.


name,population,area,ratio
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299
